# Embeddings, Conv1D en Recurrente netwerken

In [1]:
import warnings

warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

## Stap 1: Verover de data

We gebruiken de IMDB dataset met filmrecensies. Deze hebben we tijdens de bootcamp ook al eens bekeken. ter herinnering: de labels zijn binair en geven aan of een film wel of geen aanrader is volgens de reviews.

We maken het ons zelf iets moeilijker door van elke review alleen de eerste 20 woorden te gebruiken.

In [2]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20 # gebruik alleen de eerste 20 woorden van iedere review (voor efficiency)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

# gebruik alleen de eerste 20 woorden
# vul sequences korter dan 20 woorden aan met nullen
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen = maxlen)

x_val_set = x_train[:10000]
x_train_set = x_train[10000:]

y_val_set = y_train[:10000]
y_train_set = y_train[10000:]

In [3]:
print(x_train_set.shape, y_train_set.shape)

(15000, 20) (15000,)


## Stap 2: Embedding

We trainen een embedding layer in combinatie met een Dense netwerk. In onderstaande code zijn de lagen voor de embedding al ingevuld. Maak het netwerk af met een of meer dense lagen, compileer en train dit.

In [4]:
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(10000, 16, input_length = maxlen))
model.add(Flatten())
model.add(Dense(4, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            160000    
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dense (Dense)               (None, 4)                 1284      
                                                                 
 dropout (Dropout)           (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 161289 (630.04 KB)
Trainable params: 161289 (630.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# train the model
history = model.fit(x_train_set, y_train_set, epochs = 10, batch_size = 32, validation_data = (x_val_set, y_val_set))

Epoch 1/10


469/469 [==============================] - 2s 2ms/step - loss: 0.6878 - acc: 0.5361 - val_loss: 0.6718 - val_acc: 0.6580
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.6325 - acc: 0.6389 - val_loss: 0.5832 - val_acc: 0.7331
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.5577 - acc: 0.7306 - val_loss: 0.5234 - val_acc: 0.7519
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.4944 - acc: 0.7747 - val_loss: 0.5001 - val_acc: 0.7545
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.4580 - acc: 0.8008 - val_loss: 0.5012 - val_acc: 0.7523
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.4173 - acc: 0.8253 - val_loss: 0.5065 - val_acc: 0.7489
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3862 - acc: 0.8465 - val_loss: 0.5272 - val_acc: 0.7436
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 

In [6]:
# Evaluate the model
results = model.evaluate(x_test, y_test)
print(results)

782/782 [==============================] - 1s 704us/step - loss: 0.6097 - acc: 0.7271
[0.6097189784049988, 0.7271199822425842]


## Stap 3: 1D Convolutie

Experimenteer met een netwerk met een Embedding en een of meer `Conv1D` lagen.

In [7]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

model = Sequential()
model.add(Embedding(10000, 16, input_length = maxlen))
model.add(Conv1D(32, 7, activation = 'relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 3, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            160000    
                                                                 
 conv1d (Conv1D)             (None, 14, 32)            3616      
                                                                 
 max_pooling1d (MaxPooling1  (None, 4, 32)             0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 2, 32)             3104      
                                                                 
 global_max_pooling1d (Glob  (None, 32)                0         
 alMaxPooling1D)                                                 
                                                                 
 flatten_1 (Flatten)         (None, 32)              

In [8]:
# train the model
history = model.fit(x_train_set, y_train_set, epochs = 10, batch_size = 32, validation_data = (x_val_set, y_val_set))

# Evaluate the model
results = model.evaluate(x_test, y_test)


Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.6587 - acc: 0.5920 - val_loss: 0.5858 - val_acc: 0.6844
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.5257 - acc: 0.7369 - val_loss: 0.5388 - val_acc: 0.7220
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4545 - acc: 0.7884 - val_loss: 0.5485 - val_acc: 0.7230
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4043 - acc: 0.8205 - val_loss: 0.5567 - val_acc: 0.7217
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.3508 - acc: 0.8507 - val_loss: 0.5981 - val_acc: 0.7172
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2929 - acc: 0.8813 - val_loss: 0.6408 - val_acc: 0.7144
Epoch 7/10
469/469 [==============================] - 1s 3ms/step - loss: 0.2322 - acc: 0.9109 - val_loss: 0.7849 - val_acc: 0.6909
Epoch 8/10
469/469 [==============================] - 1s 3ms/step - loss: 0.

In [25]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

model = Sequential()
model.add(Embedding(10000, 16, input_length = maxlen))
model.add(Conv1D(32, 7, activation = 'relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 3, activation = 'relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dense(8, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 20, 16)            160000    
                                                                 
 conv1d_15 (Conv1D)          (None, 14, 32)            3616      
                                                                 
 dropout_12 (Dropout)        (None, 14, 32)            0         
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 4, 32)             0         
 g1D)                                                            
                                                                 
 conv1d_16 (Conv1D)          (None, 2, 32)             3104      
                                                                 
 dropout_13 (Dropout)        (None, 2, 32)             0         
                                                      

In [26]:
# train the model
history = model.fit(x_train_set, y_train_set, epochs = 8, batch_size = 16, validation_data = (x_val_set, y_val_set))

# Evaluate the model
results = model.evaluate(x_test, y_test)

Epoch 1/8
938/938 [==============================] - 3s 3ms/step - loss: 0.6816 - acc: 0.5476 - val_loss: 0.6468 - val_acc: 0.6612
Epoch 2/8
938/938 [==============================] - 3s 3ms/step - loss: 0.6064 - acc: 0.6761 - val_loss: 0.5881 - val_acc: 0.7062
Epoch 3/8
938/938 [==============================] - 3s 3ms/step - loss: 0.5573 - acc: 0.7308 - val_loss: 0.5638 - val_acc: 0.7067
Epoch 4/8
938/938 [==============================] - 3s 3ms/step - loss: 0.5325 - acc: 0.7509 - val_loss: 0.5549 - val_acc: 0.7148
Epoch 5/8
938/938 [==============================] - 3s 3ms/step - loss: 0.5109 - acc: 0.7681 - val_loss: 0.5455 - val_acc: 0.7271
Epoch 6/8
938/938 [==============================] - 3s 3ms/step - loss: 0.4892 - acc: 0.7823 - val_loss: 0.5411 - val_acc: 0.7188
Epoch 7/8
938/938 [==============================] - 3s 3ms/step - loss: 0.4790 - acc: 0.7914 - val_loss: 0.5425 - val_acc: 0.7213
Epoch 8/8
782/782 [==============================] - 1s 1ms/step - loss: 0.5429 - a

## BONUS: pre-trained embeddings